<a href="https://colab.research.google.com/github/profteachkids/CHE3022/blob/main/Pipe_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio 
from scipy.optimize import newton, root

In [2]:
def f_Churchill(Re, eD):
# Churchill, S.W. (1977). "Friction factor equation spans all fluid-flow regimes".
# Chemical engineering. 84 (24): 91–92.

    A=(2.457*np.log(1/((7/Re)**0.9+0.27*eD)))**16
    B=(37530/Re)
    return 2*((8/Re)**12+1/(A+B)**1.5)**(1/12)

In [3]:
Re = np.logspace(2.5, 8, 100)
eD_list = np.logspace(-5, -1, 12)
order = np.round(np.log10(eD_list))
eD_list = np.round(eD_list*10**(-order),1)*10**(order)

In [4]:
fig = make_subplots(rows=1, cols=1)
for eD in eD_list:
  f = f_Churchill(Re,eD)
  fig.add_scatter(x=Re, y=f)
  fig.add_annotation(x=8.3, y=np.log10(f[-1]), text=f'{eD:.1e}',ax=0, ay=0)
fig.update_layout(width=800, height=500, xaxis_type='log', yaxis_type='log', showlegend=False,
                  template=pio.templates['plotly_dark'])

In [5]:
g=9.81
pi = np.pi
rho = 1e3
mu = 1e-3
eD = 1e-3

In [6]:
def dHead(Q, L, K, D, eD):
  v = np.abs(Q / (pi*D**2/4))
  Re = rho * v * D / mu
  return -np.sign(Q)*(K+ 4*f_Churchill(Re, eD)*L/D)*v**2/2 / g

In [7]:
def flow(deltaHead, L, K, D, eD):

  abs_deltaHead=np.abs(deltaHead)
  if abs_deltaHead<1e-10:
    return 0.0
  Q_guess = pi*D**2/4 * np.sqrt(abs_deltaHead*2*g/(K+4*(0.01)*L/D))

  def f_to_zero(Q):
    return dHead(Q, L, K, D, eD) - abs_deltaHead

  res = newton(f_to_zero, Q_guess)
  return np.sign(deltaHead)*res

In [8]:
# array of pipes, fromNode, toNode, length, K
pipes = np.array([[0,1, 1.5, 1.5],
                  [0,2, 2.0, 2.0],
                  [1,2, 1.5, 0.0],
                  [1,3, 1.5, 0.0],
                  [2,4, 0.5, 0.0],
                  [3,4, 3.0, 2.0]])

z = np.array([30.0, 0.0, 0.0, 0.0 ,0.0])
externalP_indices = [0, 4]
externalP = np.array([2e5, 0.0])

fromNode = pipes[:,0].astype(np.int32)
toNode = pipes[:,1].astype(np.int32)
lengths = pipes[:,2]
K = pipes[:,3]

n_Pipes = pipes.shape[0]
pipeD = np.full(n_Pipes, 0.1)
pipe_eD = np.full(n_Pipes, 1e-4)
n_Nodes = np.max(pipes[:,0:2]).astype(np.int32)+1

internal_indices = list(set(np.arange(n_Nodes).astype(np.int32))-set(externalP_indices))
n_internal_indices = len(internal_indices)

flows=np.zeros(n_Pipes)
netQ = np.zeros(n_Nodes)
P = np.zeros(n_Nodes)

In [9]:

heads = np.zeros(n_Nodes)
P[externalP_indices] = externalP

avgExternalHead = np.mean(externalP/rho/g + z[externalP_indices])
internalHeadGuess = np.full(n_internal_indices, avgExternalHead)
internalP_guess = (internalHeadGuess - z[internal_indices])*rho*g

In [10]:
P

array([200000.,      0.,      0.,      0.,      0.])

In [11]:
def Q_to_zero(internalP):
  global heads, netQ, flows, P
  P[internal_indices]=internalP
  heads = P/rho/g + z

  for i in range(n_Pipes):
    deltaHead = heads[toNode[i]] - heads[fromNode[i]]
    flows[i]= flow(deltaHead, lengths[i], K[i], pipeD[i], pipe_eD[i])

  netQ.fill(0.0)
  np.add.at(netQ, toNode, flows)
  np.add.at(netQ, fromNode, -flows)
  
  return netQ[internal_indices]

In [12]:
res = root(Q_to_zero, internalP_guess)

In [13]:
for i in range(n_Nodes):
  print(f'netQ {i}: {netQ[i]:9.5f}   P:{P[i]:9.0f}')

print()
for i in range(n_Pipes):
  print(f'{fromNode[i]} to {toNode[i]} : {flows[i]:9.5f} m3/s')

netQ 0:  -0.33478   P:   200000
netQ 1:   0.00000   P:    63719
netQ 2:  -0.00000   P:    39869
netQ 3:   0.00000   P:    58641
netQ 4:   0.33478   P:        0

0 to 1 :   0.17719 m3/s
0 to 2 :   0.15759 m3/s
1 to 2 :   0.12250 m3/s
1 to 3 :   0.05469 m3/s
2 to 4 :   0.28009 m3/s
3 to 4 :   0.05469 m3/s
